In [1]:
def switch_left():
    ############## iframe으로 왼쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="searchIframe"]')
    driver.switch_to.frame(iframe)
    
def switch_right():
    ############## iframe으로 오른쪽 포커스 맞추기 ##############
    driver.switch_to.parent_frame()
    iframe = driver.find_element(By.XPATH,'//*[@id="entryIframe"]')
    driver.switch_to.frame(iframe)

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep
import random
import re
import pandas as pd

from selenium import webdriver
import sys

name_li = []
address_li = []
rating_li = []
review_li = []
var_1 = 0

options = webdriver.ChromeOptions()
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')
options.add_argument('window-size=1380,900')
driver = webdriver.Chrome(options=options)

# 대기 시간
driver.implicitly_wait(time_to_wait=3)

# 반복 종료 조건
loop = True

URL = 'https://map.naver.com/p/search/%EC%A0%9C%EC%A3%BC%EB%8F%84%20%EC%B9%B4%ED%8E%98?c=9.00,0,0,0,dh'
driver.get(url=URL)

while(loop):
    switch_left()

    # 페이지 숫자를 초기에 체크 [ True / False ]
    # 이건 페이지 넘어갈때마다 계속 확인해줘야 함 (페이지 새로 로드 될때마다 버튼 상태 값이 바뀜)
    next_page = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[7]').get_attribute('aria-disabled')
    
    if(next_page == 'true'):
        break

    ############## 맨 밑까지 스크롤 ##############
    scrollable_element = driver.find_element(By.CLASS_NAME, "Ryr1F")

    last_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

    while True:
        # 요소 내에서 아래로 600px 스크롤
        driver.execute_script("arguments[0].scrollTop += 600;", scrollable_element)

        # 페이지 로드를 기다림
        sleep(1)  # 동적 콘텐츠 로드 시간에 따라 조절

        # 새 높이 계산
        new_height = driver.execute_script("return arguments[0].scrollHeight", scrollable_element)

        # 스크롤이 더 이상 늘어나지 않으면 루프 종료
        if new_height == last_height:
            break

        last_height = new_height


    ############## 현재 page number 가져오기 - 1 페이지 ##############

    page_no = driver.find_element(By.XPATH,'//a[contains(@class, "mBN2s qxokY")]').text

    # 현재 페이지에 등록된 모든 가게 조회
    # 첫페이지 광고 2개 때문에 첫페이지는 앞 2개를 빼야함
    if(page_no == '1'):
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')[2:]
    else:
        elemets = driver.find_elements(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]//li')

    print('현재 ' + '\033[95m' + str(page_no) + '\033[0m' + ' 페이지 / '+ '총 ' + '\033[95m' + str(len(elemets)) + '\033[0m' + '개의 가게를 찾았습니다.\n')

    for index, e in enumerate(elemets, start=1):
        sleep(1)
        final_element = e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span")

        print(str(index) + ". " + final_element.text)

    # print(Colors.RED + "-"*50 + Colors.RESET)

    switch_left()

    sleep(1.5)

    for index, e in enumerate(elemets, start=1):
        store_name = '' # 가게 이름
        category = '' # 카테고리
        new_open = '' # 새로 오픈
        
        rating = 0.0 # 평점
        visited_review = 0 # 방문자 리뷰
        blog_review = 0 # 블로그 리뷰
        store_id = '' # 가게 고유 번호
        
        address = '' # 가게 주소
        business_hours = [] # 영업 시간
        phone_num = '' # 전화번호

        switch_left()


        # 순서대로 값을 하나씩 클릭
        e.find_element(By.CLASS_NAME,'CHC5F').find_element(By.XPATH, ".//a/div/div/span").click()

        sleep(1.5)

        switch_right()

        ################### 여기부터 크롤링 시작 ##################
        title = driver.find_element(By.XPATH,'//div[@class="zD5Nm undefined"]')
        store_info = title.find_elements(By.XPATH,'//div[@class="YouOG DZucB"]/div/span')


        # 가게 이름
        store_name = title.find_element(By.XPATH,'.//div[1]/div[1]/span[1]').text

        # 카테고리
        category = title.find_element(By.XPATH,'.//div[1]/div[1]/span[2]').text

        if(len(store_info) > 2):
            # 새로 오픈
            new_open = title.find_element(By.XPATH,'.//div[1]/div[1]/span[3]').text


        ###############################

        review = title.find_elements(By.XPATH,'.//div[2]/span')

        # 인덱스 변수 값
        _index = 1

        # 리뷰 ROW의 갯수가 3개 이상일 경우 [별점, 방문자 리뷰, 블로그 리뷰]
        if len(review) > 2:
            rating_xpath = f'.//div[2]/span[{_index}]'
            rating_element = title.find_element(By.XPATH, rating_xpath)
            rating = rating_element.text.replace("\n", " ")

            _index += 1

        try:
            # 방문자 리뷰
            visited_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text


            # 인덱스를 다시 +1 증가 시킴
            _index += 1

            # 블로그 리뷰
            blog_review = title.find_element(By.XPATH,f'.//div[2]/span[{_index}]/a').text
        except:
            print('------------ 리뷰 부분 오류 ------------')

        # 가게 id
        store_id = driver.find_element(By.XPATH,'//div[@class="flicking-camera"]/a').get_attribute('href').split('/')[4]







        # 가게 주소
        address = driver.find_element(By.XPATH,'//span[@class="LDgIH"]').text


        try:
            driver.find_element(By.XPATH,'//div[@class="y6tNq"]//span').click()

            # 영업 시간 더보기 버튼을 누르고 2초 반영시간 기다림
            sleep(1.5)


            parent_element = driver.find_element(By.XPATH,'//a[@class="gKP9i RMgN0"]')
            child_elements = parent_element.find_elements(By.XPATH, './*[@class="w9QyJ" or @class="w9QyJ undefined"]')

            for child in child_elements:
                # 각 자식 요소 내에서 클래스가 'A_cdD'인 span 요소 찾기
                span_elements = child.find_elements(By.XPATH, './/span[@class="A_cdD"]')

                # 찾은 span 요소들의 텍스트 출력
                for span in span_elements:
                    business_hours.append(span)
            
            # 가게 전화번호
            phone_num = driver.find_element(By.XPATH,'//span[@class="xlx7Q"]').text

        except:
            print(print('------------ 영업시간 / 전화번호 부분 오류 ------------'))
        
        name_li.append(store_name)
        address_li.append(address)
        rating_li.append(rating)
        review_li.append(visited_review)
        print(f'{index}. ' + str(store_name) + ' · ' + str(category) + str(new_open))
        print('평점 ' + str(rating) + ' / ' + visited_review + ' · ' + blog_review)
        print(f'가게 고유 번호 -> {store_id}')
        print('가게 주소 ' + str(address))
        print('가게 영업 시간')
        for i in business_hours:
            print(i.text)
            print('')
        print('가게 번호 ' + phone_num)
        # print(Colors.MAGENTA + "-"*50 + Colors.RESET)

    switch_left()
    if(var_1 == 1):
        loop = False  
    # 페이지 다음 버튼이 활성화 상태일 경우 계속 진행
    if(next_page == 'false'):
        driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[7]').click()
    # 아닐 경우 루프 정지
    else:
        var_1 = 1
        # loop = False



현재 1 페이지 / 총 52개의 가게를 찾았습니다.

1. 런던베이글뮤지엄 제주점
2. 해지개
3. 원앤온리
4. 아베베베이커리
5. 호텔샌드
6. 노을리
7. 새물
8. 아줄레주
9. 오드랑베이커리
10. 랜디스도넛 제주애월점
11. 웨이뷰 협재바다유채꽃
12. 더클리프
13. 스타벅스 더제주송당파크R점
14. 우무
15. 인스밀
16. 몽그레 제주공항점
17. 바다다
18. 애월빵공장
19. 목장카페 밭디
20. 서귀피안 베이커리
21. 아리나 블루문
22. 파인밀 베이커리카페 제주점
23. 카페루시아 본점
24. 무로이 신화월드유채꽃
25. 커피스케치
26. 포레스트제이 카우셰드
27. 델문도
28. 점점
29. 찰리공장 제주 본점
30. 제주바솔트
31. 파리바게뜨 제주국제공항점
32. 찌즈
33. 소색채본
34. 해일리 카페
35. 카페아오오
36. 유동커피
37. 테라로사 서귀포점
38. 리부아르
39. 휴즐리 제주
40. 우무
41. 바다바라 베이커리 카페
42. 오른
43. 스타벅스 제주중문점
44. 카페진정성 종점
45. 무로이제과점 선물가게야시장탑동
46. 새빌
47. 프릳츠 제주성산점
48. 앙뚜아네트
49. 노티드 제주
50. 제주 하멜
51. 제주만주
52. 블루보틀 제주 카페
------------ 영업시간 / 전화번호 부분 오류 ------------
None
1. 런던베이글뮤지엄 제주점 · 베이커리
평점 0.0 / 방문자리뷰 2,965 · 블로그리뷰 4,415
가게 고유 번호 -> 1422544447
가게 주소 제주 제주시 구좌읍 동복로 85 제2동 1층
가게 영업 시간
매일
08:00 - 18:00

가게 번호 
2. 해지개 · 카페,디저트
평점 0.0 / 방문자리뷰 2,401 · 블로그리뷰 2,720
가게 고유 번호 -> 1412199535
가게 주소 제주 제주시 애월읍 애월북서길 52 . 1~2층
가게 영업 시간
매일
09:00 - 21:00
20:20 라스트오더

가게 번호 0507-1358-8586
3. 원앤온리 · 카페


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.123)
Stacktrace:
	GetHandleVerifier [0x00007FF6BF277032+63090]
	(No symbol) [0x00007FF6BF1E2C82]
	(No symbol) [0x00007FF6BF07EC65]
	(No symbol) [0x00007FF6BF05CA7C]
	(No symbol) [0x00007FF6BF0ED687]
	(No symbol) [0x00007FF6BF102AC1]
	(No symbol) [0x00007FF6BF0E6D83]
	(No symbol) [0x00007FF6BF0B83A8]
	(No symbol) [0x00007FF6BF0B9441]
	GetHandleVerifier [0x00007FF6BF6725AD+4238317]
	GetHandleVerifier [0x00007FF6BF6AF70D+4488525]
	GetHandleVerifier [0x00007FF6BF6A79EF+4456495]
	GetHandleVerifier [0x00007FF6BF350576+953270]
	(No symbol) [0x00007FF6BF1EE54F]
	(No symbol) [0x00007FF6BF1E9224]
	(No symbol) [0x00007FF6BF1E935B]
	(No symbol) [0x00007FF6BF1D9B94]
	BaseThreadInitThunk [0x00007FFAE52D7344+20]
	RtlUserThreadStart [0x00007FFAE72826B1+33]


In [5]:
total_df = pd.DataFrame({
    'Name': name_li,
    'Address' : address_li,
    'Rating' : rating_li,
    'Review' : review_li
})

In [36]:
# 별점 문구 삭제 코드
total_df['Rating'] = [float(str(x).split()[-1]) for x in total_df.Rating]
# type(total_df.Rating[1])

# 방문자리뷰 문구 삭제 코드
total_df['Review'] = [float(x.split()[-1].replace(',','')) for x in total_df.Review]
# type(total_df.Review[1])



numpy.float64

In [ ]:
total_df

In [ ]:
# 읍면동 추출해서 컬럼에 추가하기
# total_df['mid_addr'] = [x.split()[] for x in total_df.Address]

In [18]:
import pickle

# with open('./jeju_1.pkl', 'wb') as f:
#     pickle.dump(total_df, f)
with open('./jeju_eumsikjeom.pkl', 'rb') as f:
    total2 = pickle.load(f)

In [19]:
total2

,Name,Address,Rating,Review
0,해녀촌,제주 제주시 구좌읍 동복로 33,4.34,1992.0
1,우진해장국,제주 제주시 서사로 11,4.41,20604.0
2,황해식당 제주공항 갈치,제주 제주시 일주서로 7682 황해식당 제주공항 갈치,4.70,3256.0
3,대담 애월본점,제주 제주시 애월읍 일주서로 7035 대담,0.00,945.0
4,이춘옥원조고등어쌈밥 제주애월본점,제주 제주시 애월읍 일주서로 7213,0.00,16794.0
...,...,...,...,...
247,갈치음식점 궁서체 제주탑동본점,제주 제주시 탑동로11길 6 2층 궁서체,0.00,1385.0
248,모해통갈치화덕구이,제주 서귀포시 성산읍 일출로 267 모해통갈치화덕구이,0.00,1877.0
249,가로수길마루 애월 본점,제주 제주시 애월읍 애월로 11 가로수길마루,4.74,491.0
250,미친부엌 제주본점,"제주 제주시 탑동로 15 1, 2층",4.48,2239.0


In [5]:
import pickle
with open('./jeju_cafe.pkl', 'rb') as f:
    cafe_df = pickle.load(f)


In [13]:
cafe_df

,store_name,address,rating,visited_review
0,런던베이글뮤지엄 제주점,제주 제주시 구좌읍 동복로 85 제2동 1층,0.00,2965.0
1,해지개,제주 제주시 애월읍 애월북서길 52 . 1~2층,0.00,2401.0
2,원앤온리,제주 서귀포시 안덕면 산방로 141,0.00,13102.0
3,아베베베이커리,제주 제주시 동문로6길 4 동문시장 12번 게이트 옆,4.60,10480.0
4,호텔샌드,제주 제주시 한림읍 한림로 339,4.39,2794.0
...,...,...,...,...
247,덕인당보리빵 제주점,제주 제주시 연삼로 180,4.31,761.0
248,팀블로우,제주 제주시 애월읍 애월로 19-5 1층,4.58,1463.0
249,러디스 서귀포본점,제주 서귀포시 월드컵로 202 제주 월드컵로 202,4.36,1120.0
250,무우수 커피 로스터스,제주 제주시 조천읍 조천11길 22-2 1F,0.00,245.0


In [12]:
cafe_df['visited_review'] = [float(str(x).split()[-1].replace(',','')) for x in cafe_df.visited_review]


In [14]:
cafe_df.rename(columns={'store_name':"Name", 'address':'Address', 'visited_review':'Review', 'rating':'Rating'},inplace=True)

In [15]:
cafe_df

,Name,Address,Rating,Review
0,런던베이글뮤지엄 제주점,제주 제주시 구좌읍 동복로 85 제2동 1층,0.00,2965.0
1,해지개,제주 제주시 애월읍 애월북서길 52 . 1~2층,0.00,2401.0
2,원앤온리,제주 서귀포시 안덕면 산방로 141,0.00,13102.0
3,아베베베이커리,제주 제주시 동문로6길 4 동문시장 12번 게이트 옆,4.60,10480.0
4,호텔샌드,제주 제주시 한림읍 한림로 339,4.39,2794.0
...,...,...,...,...
247,덕인당보리빵 제주점,제주 제주시 연삼로 180,4.31,761.0
248,팀블로우,제주 제주시 애월읍 애월로 19-5 1층,4.58,1463.0
249,러디스 서귀포본점,제주 서귀포시 월드컵로 202 제주 월드컵로 202,4.36,1120.0
250,무우수 커피 로스터스,제주 제주시 조천읍 조천11길 22-2 1F,0.00,245.0


In [16]:
import sqlalchemy
from urllib import parse

user = 'encore5'
password = '' # blind
host='' # blind
port = 3306
database = 'encore5'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")



cafe_df.to_sql('jejudo_cafe', index=False, if_exists="append", con=engine)


252

In [33]:
import pandas as pd
import re
p = re.compile('\d-\d|\d')
jeju_df = pd.read_csv('./jejudo_202404180949.csv')

address_ch = [x.split() for x in jeju_df.Address]
new_addr = []
for l in address_ch:
    addr = ''
    # l은 ['제주', '제주시', '구좌읍', '동복로', '85', '제2동', '1층']와 같은 리스트
    for s in l:
        addr += ' ' + s
        # 정규식 사용해서 정수나 정수-정수 형태인 경우 문자열 합치기 멈춤
        if p.match(s) != None:
            print(s)
            break
    new_addr.append(addr.strip())



85
52
141
4
339
654
620
59
552-3
12
25-3
154-17
1197
542-1
22
84
148-15
42-15
2486
17
74
3
49-17
21
24-32
16
519-10
2384
21
6
2
7
24-61
269-37
75
406-1
27-16
686
83
40-1
29-51
2714
32
124
15
1529
222
671
24-9
51-3
6707
2133-30
45-1
45
56
400
12
16
93
783
45-1
58-1
291-5
49-65
48
228-13
2610
19-8
39-2
359
207-2
624
1647
22
4611
44
16
27-2
30
4492
96
178
2267-4
29-1
13-7
56
3
2
80
376
191-43
550
412
11
39
2
60
65
9-16
174
220-1
468-2
160-20
6111
48-1
579
380
21
11
230
2064
964-2
198-5
45
4
454-3
33
56
655
1611
24
269
20
40
9-2
140
45-1
6
87
448
522
3
6-6
84
2783
494
36
57
25
46-16
144
24-15
24-8
30
183-16
6
16-14
11
564
6
26
85
239
162
810
26
1785-12
29-9
552-22
337
48-2
29
272
48-59
176
474
284-5
114
1166
95
264-6
354
1132
141
38
132
780-2
34
9
9
9
36-1
84
27
86
85-13
684
11
45-3
83
5855
35
5023
32
120
24
29-10
648
15
1199
218
17
129
37
112
272
4772
66
22
20-1
792
64
77-3
4065-1
442-1
444-1
45
16
2
122
57
23
30-8
28
21
84
32
63
30
133
393
1218
29
201
6
13
990
36-1
2-2
239
13
8
337
27-7


In [34]:
new_addr

['제주 제주시 구좌읍 동복로 85',
 '제주 제주시 애월읍 애월북서길 52',
 '제주 서귀포시 안덕면 산방로 141',
 '제주 제주시 동문로6길 4',
 '제주 제주시 한림읍 한림로 339',
 '제주 제주시 애월읍 애월해안로 654',
 '제주 제주시 애월읍 애월해안로 620',
 '제주 서귀포시 성산읍 신풍하동로19번길 59',
 '제주 제주시 조천읍 조함해안로 552-3',
 '제주 제주시 애월읍 애월로 12',
 '제주 제주시 한림읍 옹포7길 25-3',
 '제주 서귀포시 중문관광로 154-17',
 '제주 제주시 구좌읍 비자림로 1197',
 '제주 제주시 한림읍 한림로 542-1',
 '제주 서귀포시 대정읍 일과대수로27번길 22',
 '제주 제주시 도리로 84',
 '제주 서귀포시 대포로 148-15',
 '제주 제주시 애월읍 금성5길 42-15',
 '제주 서귀포시 표선면 번영로 2486',
 '제주 서귀포시 성산읍 신양로122번길 17',
 '제주 제주시 애월읍 하귀로 74',
 '제주 제주시 구좌읍 종달항길 3',
 '제주 서귀포시 안덕면 난드르로 49-17',
 '제주 서귀포시 안덕면 동광본동로 21',
 '제주 서귀포시 안덕면 사계남로216번길 24-32',
 '제주 서귀포시 안덕면 화순서서로64번길 16',
 '제주 제주시 조천읍 조함해안로 519-10',
 '제주 제주시 조천읍 신촌리 2384',
 '제주 제주시 중앙로13길 21',
 '제주 제주시 가령골3길 6',
 '제주 제주시 공항로 2',
 '제주 제주시 서문로5길 7',
 '제주 서귀포시 안덕면 사계남로216번길 24-61',
 '제주 서귀포시 성산읍 한도로 269-37',
 '제주 서귀포시 성산읍 환해장성로 75',
 '제주 서귀포시 태평로 406-1',
 '제주 서귀포시 칠십리로658번길 27-16',
 '제주 제주시 애월읍 애월해안로 686',
 '제주 제주시 흥운길 83',
 '제주 제주시 관덕로8길 40-1',
 '제주 서귀포시 중문관광로72번길 29-51',

In [66]:
import requests
from bs4 import BeautifulSoup as BS

old_addr = []
for a in new_addr:
    url = f'https://www.juso.go.kr/support/AddressMainSearch.do?searchKeyword={a}'
    r = requests.get(url)
# url = 'https://www.juso.go.kr/support/AddressMainSearch.do?searchKeyword=제주 제주시 도리로 84'
# r = requests.get(url)    
    print(a)
    try:
        old_addr.append(BS(r.text).find_all('span',class_='roadNameText')[1].text.strip().replace('  ', ' '))
    except:
        old_addr.append(a)
# 정규식을 이용하여 제어 문자 삭제
old_addr = [re.sub('[\t\r\n]','',x) for x in old_addr]

제주 제주시 구좌읍 동복로 85
제주 제주시 애월읍 애월북서길 52
제주 서귀포시 안덕면 산방로 141
제주 제주시 동문로6길 4
제주 제주시 한림읍 한림로 339
제주 제주시 애월읍 애월해안로 654
제주 제주시 애월읍 애월해안로 620
제주 서귀포시 성산읍 신풍하동로19번길 59
제주 제주시 조천읍 조함해안로 552-3
제주 제주시 애월읍 애월로 12
제주 제주시 한림읍 옹포7길 25-3
제주 서귀포시 중문관광로 154-17
제주 제주시 구좌읍 비자림로 1197
제주 제주시 한림읍 한림로 542-1
제주 서귀포시 대정읍 일과대수로27번길 22
제주 제주시 도리로 84
제주 서귀포시 대포로 148-15
제주 제주시 애월읍 금성5길 42-15
제주 서귀포시 표선면 번영로 2486
제주 서귀포시 성산읍 신양로122번길 17
제주 제주시 애월읍 하귀로 74
제주 제주시 구좌읍 종달항길 3
제주 서귀포시 안덕면 난드르로 49-17
제주 서귀포시 안덕면 동광본동로 21
제주 서귀포시 안덕면 사계남로216번길 24-32
제주 서귀포시 안덕면 화순서서로64번길 16
제주 제주시 조천읍 조함해안로 519-10
제주 제주시 조천읍 신촌리 2384
제주 제주시 중앙로13길 21
제주 제주시 가령골3길 6
제주 제주시 공항로 2
제주 제주시 서문로5길 7
제주 서귀포시 안덕면 사계남로216번길 24-61
제주 서귀포시 성산읍 한도로 269-37
제주 서귀포시 성산읍 환해장성로 75
제주 서귀포시 태평로 406-1
제주 서귀포시 칠십리로658번길 27-16
제주 제주시 애월읍 애월해안로 686
제주 제주시 흥운길 83
제주 제주시 관덕로8길 40-1
제주 서귀포시 중문관광로72번길 29-51
제주 서귀포시 성산읍 해맞이해안로 2714
제주 서귀포시 중문관광로110번길 32
제주 제주시 서해안로 124
제주 제주시 동문로2길 15
제주 제주시 애월읍 평화로 1529
제주 서귀포시 성산읍 일출로 222
제주 제주시 서해안로 671
제주 제주시 애월읍 애월로1길 24-9
제주 제주시 

In [75]:
jeju_df['oldAddr'] = old_addr

In [77]:
mid_addr = [x.split()[2] for x in old_addr]

In [78]:
mid_addr

['구좌읍',
 '애월읍',
 '안덕면',
 '일도일동',
 '한림읍',
 '애월읍',
 '애월읍',
 '성산읍',
 '조천읍',
 '애월읍',
 '한림읍',
 '색달동',
 '구좌읍',
 '한림읍',
 '대정읍',
 '이호일동',
 '대포동',
 '애월읍',
 '표선면',
 '성산읍',
 '애월읍',
 '구좌읍',
 '안덕면',
 '안덕면',
 '안덕면',
 '안덕면',
 '조천읍',
 '조천읍',
 '일도일동',
 '이도이동',
 '용담이동',
 '용담일동',
 '안덕면',
 '성산읍',
 '성산읍',
 '서귀동',
 '하효동',
 '애월읍',
 '용담이동',
 '삼도이동',
 '색달동',
 '성산읍',
 '색달동',
 '이호일동',
 '일도일동',
 '애월읍',
 '성산읍',
 '용담삼동',
 '애월읍',
 '노형동',
 '애월읍',
 '구좌읍',
 '이도이동',
 '일도일동',
 '남원읍',
 '구좌읍',
 '안덕면',
 '애월읍',
 '애월읍',
 '우도면',
 '애월읍',
 '한림읍',
 '도두이동',
 '안덕면',
 '구좌읍',
 '토평동',
 '표선면',
 '한림읍',
 '성산읍',
 '한림읍',
 '애월읍',
 '용담삼동',
 '구좌읍',
 '한림읍',
 '한경면',
 '구좌읍',
 '대정읍',
 '오라일동',
 '안덕면',
 '한경면',
 '내도동',
 '보목동',
 '구좌읍',
 '일도일동',
 '호근동',
 '조천읍',
 '이도이동',
 '용담이동',
 '성산읍',
 '애월읍',
 '안덕면',
 '봉개동',
 '대정읍',
 '삼양일동',
 '구좌읍',
 '용담이동',
 '한림읍',
 '연동',
 '노형동',
 '대포동',
 '대정읍',
 '용담삼동',
 '월평동',
 '애월읍',
 '성산읍',
 '용담삼동',
 '용담삼동',
 '구좌읍',
 '애월읍',
 '성산읍',
 '구좌읍',
 '남원읍',
 '조천읍',
 '애월읍',
 '도두일동',
 '애월읍',
 '애월읍',
 '구좌읍',
 '하효동',
 '조천읍',
 '오라삼동',
 '월평동',

In [79]:
jeju_df['midAddr'] = mid_addr

In [83]:
jeju_df

,Name,Address,Rating,Review,oldAddr,midAddr
0,런던베이글뮤지엄 제주점,제주 제주시 구좌읍 동복로 85 제2동 1층,0.00,2965.0,제주특별자치도 제주시 구좌읍 동복리 719-1,구좌읍
1,해지개,제주 제주시 애월읍 애월북서길 52 . 1~2층,0.00,2401.0,제주특별자치도 제주시 애월읍 애월리 2555,애월읍
2,원앤온리,제주 서귀포시 안덕면 산방로 141,0.00,13102.0,제주특별자치도 서귀포시 안덕면 사계리 86 원앤온리,안덕면
3,아베베베이커리,제주 제주시 동문로6길 4 동문시장 12번 게이트 옆,4.60,10480.0,제주특별자치도 제주시 일도일동 1136-2,일도일동
4,호텔샌드,제주 제주시 한림읍 한림로 339,4.39,2794.0,제주특별자치도 제주시 한림읍 협재리 2446-3,한림읍
...,...,...,...,...,...,...
473,가로수길마루 애월 본점,제주 제주시 애월읍 애월로 11 가로수길마루,4.74,491.0,제주특별자치도 제주시 애월읍 애월리 2467,애월읍
474,미친부엌 제주본점,"제주 제주시 탑동로 15 1, 2층",4.48,2239.0,제주특별자치도 제주시 삼도이동 1192-28,삼도이동
475,미향해장국 성산점,제주 서귀포시 성산읍 일출로 9 1층,0.00,3073.0,제주특별자치도 서귀포시 성산읍 고성리 1098-1,성산읍
476,포르뚜나,제주 제주시 한림읍 한림해안로 136 1층 카페 포르뚜나,0.00,46.0,제주특별자치도 제주시 한림읍 한림리 1587,한림읍


In [84]:
jeju_df.midAddr.unique()

array(['구좌읍', '애월읍', '안덕면', '일도일동', '한림읍', '성산읍', '조천읍', '색달동', '대정읍',
       '이호일동', '대포동', '표선면', '이도이동', '용담이동', '용담일동', '서귀동', '하효동', '삼도이동',
       '용담삼동', '노형동', '남원읍', '우도면', '도두이동', '토평동', '한경면', '오라일동', '내도동',
       '보목동', '호근동', '봉개동', '삼양일동', '연동', '월평동', '도두일동', '오라삼동', '중문동',
       '이도일동', '아라일동', '강정동', '동홍동', '삼도일동', '건입동', '회수동', '상예동', '서호동',
       '법환동', '오등동', '아라이동', '삼양삼동', '오라이동', '이호이동', '하예동', '일도이동', '하원동'],
      dtype=object)

In [81]:
jeju_df.to_sql('jejudo_2', index=False, if_exists="append", con=engine)


478

In [3]:
import pandas as pd

In [82]:
with open('./jejudo_oldaddr.pkl', 'wb') as f:
    pickle.dump(jeju_df, f)

In [90]:
past_freq.to_csv('./past_freq.csv',index=False,encoding='utf-8')

In [4]:
past_freq2 = pd.read_csv('./past_freq2.csv', encoding='utf-8')
present_freq2 = pd.read_csv('./present_freq2.csv', encoding='utf-8')

In [107]:
area_freq2 = pd.concat([past_freq2, present_freq2],axis=0)

In [108]:
area_freq2

,midAddr,ratio,direction,time
0,건입동,1.394362,북동,과거
1,구좌읍,3.680382,북동,과거
2,남원읍,1.359217,남동,과거
3,노형동,5.930925,북서,과거
4,대륜동,1.057909,남서,과거
...,...,...,...,...
38,하효동,0.645185,남동,현재
39,한경면,0.795018,북서,현재
40,한림읍,6.898358,북서,현재
41,호근동,0.219765,남서,현재


In [14]:
wc_list = []
for i in range(len(present_freq2)):
    for j in range(int(present_freq2.ratio[i]*10)):
        print(present_freq2.midAddr[i], end = ' ')
    


강정동 강정동 강정동 강정동 강정동 강정동 강정동 강정동 강정동 강정동 강정동 건입동 건입동 건입동 건입동 건입동 건입동 건입동 건입동 건입동 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 구좌읍 남원읍 남원읍 남원읍 남원읍 남원읍 남원읍 남원읍 내도동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 노형동 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대정읍 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 대포동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 도두동 동홍동 법환동 보목동 보목동 보목동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼도동 삼양동 상예동 상예동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 색달동 서귀동 

In [110]:
import sqlalchemy
from urllib import parse

user = 'encore5'
password = '' # blind
host='' # blind
port = 3306
database = 'encore5'
password = parse.quote_plus(password)
engine = sqlalchemy.create_engine(f"mysql://{user}:{password}@{host}:{port}/{database}")

area_freq2.to_sql('area_freq', index=False, if_exists="append", con=engine)


82